In [1]:
from deepface import DeepFace
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from generate_dataset import generate_dataset

In [2]:
generate_dataset()

In [3]:
df = pd.read_csv("dataset.csv")
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,504,505,506,507,508,509,510,511,Celeb_Label,Celeb_name
0,0,-0.130630,0.050372,-0.033286,-0.352186,-0.064046,-0.108651,0.265626,0.004132,-0.018946,...,0.199065,-0.006498,0.083920,-0.105203,0.326752,0.145737,0.191030,0.057138,0,Adel_FerdowsiPour
1,1,-0.196588,0.239753,-0.012986,-0.313293,0.137059,-0.082240,0.229596,0.122506,-0.041383,...,0.141603,0.118120,0.337488,-0.100415,0.078088,0.118227,-0.097745,0.021733,0,Adel_FerdowsiPour
2,2,0.007586,-0.045209,0.279231,-0.122467,-0.047737,-0.093136,0.020060,-0.013580,0.078798,...,0.066340,-0.069833,0.220127,-0.079769,0.369212,0.031541,0.209013,0.324446,0,Adel_FerdowsiPour
3,3,-0.018304,-0.047322,0.070936,-0.226148,0.082088,-0.079673,0.006830,-0.121529,-0.061033,...,0.038184,0.021753,-0.011950,0.096359,0.031626,0.038657,-0.035625,-0.018383,0,Adel_FerdowsiPour
4,4,-0.134411,0.177214,0.148634,-0.626351,0.068255,-0.306762,0.124626,0.134459,-0.244582,...,0.183643,-0.025073,-0.005875,0.015631,0.247400,0.106732,-0.200120,0.002185,0,Adel_FerdowsiPour


In [4]:
celebs = df[["Celeb_name"]].drop_duplicates().reset_index(drop=True)
celebs

,Celeb_name
0,Adel_FerdowsiPour
1,Ali_Daei
2,Ali_Khamenei


In [5]:
X = df.drop(["Unnamed: 0", "Celeb_Label", "Celeb_name"],axis=1).values
Y = df["Celeb_Label"]
Y = np.eye(30)[Y]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((120, 512), (30, 512), (120, 30), (30, 30))

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation="sigmoid"),
    tf.keras.layers.Dense(750, activation="relu"),
    tf.keras.layers.Dense(400, activation="relu"),
    tf.keras.layers.Dense(30, activation="softmax")
    ])

In [10]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.binary_crossentropy,
              metrics = ["accuracy"])

In [11]:
model.fit(X_train, Y_train, epochs=50)

Epoch 1/50
4/4 [==============================] - 1s 12ms/step - loss: 0.4275 - accuracy: 0.0583
Epoch 2/50
4/4 [==============================] - 0s 12ms/step - loss: 0.0816 - accuracy: 0.4083
Epoch 3/50
4/4 [==============================] - 0s 12ms/step - loss: 0.0634 - accuracy: 0.4667
Epoch 4/50
4/4 [==============================] - 0s 10ms/step - loss: 0.0577 - accuracy: 0.5667
Epoch 5/50
4/4 [==============================] - 0s 11ms/step - loss: 0.0530 - accuracy: 0.6000
Epoch 6/50
4/4 [==============================] - 0s 11ms/step - loss: 0.0439 - accuracy: 0.7333
Epoch 7/50
4/4 [==============================] - 0s 10ms/step - loss: 0.0369 - accuracy: 0.8167
Epoch 8/50
4/4 [==============================] - 0s 10ms/step - loss: 0.0320 - accuracy: 0.9333
Epoch 9/50
4/4 [==============================] - 0s 14ms/step - loss: 0.0239 - accuracy: 0.9917
Epoch 10/50
4/4 [==============================] - 0s 10ms/step - loss: 0.0172 - accuracy: 0.9917
Epoch 11/50
4/4 [============

In [ ]:
model.save("model.h5")

In [7]:
model = tf.keras.models.load_model("model.h5")

In [8]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 248ms/step - loss: 1.4044e-04 - accuracy: 1.0000


[0.00014044200361240655, 1.0]

In [10]:
embedding_objs = DeepFace.represent(img_path = "images/Ali_Daei/279746_827-0.jpg", model_name="ArcFace")
predict = model.predict(np.array([embedding_objs[0]["embedding"]]))
celebs["Celeb_name"][np.argmax(predict)]

1/1 [==============================] - 0s 19ms/step


'Ali_Daei'